In [4]:

import os

In [5]:
%pwd

'/Volumes/RRR/Macbook-workdrive/DataScienceEnd2EndProjects/Chicken-Disease-Classification/research'

In [6]:
os.chdir("../")
%pwd

'/Volumes/RRR/Macbook-workdrive/DataScienceEnd2EndProjects/Chicken-Disease-Classification'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'cnnClassifier'